In [1]:
import pandas as pd
import requests
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
headers = {'User-Agent': "your email address"}

In [3]:
# API call to get mapping between CIK and Tickers  
tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)

In [5]:
# JSON object to Pandas DF
tickers_cik = pd.json_normalize(pd.json_normalize(tickers_cik.json(),\
max_level=0).values[0])
tickers_cik["cik_str"] = tickers_cik["cik_str"].astype(str).str.zfill(10)
tickers_cik.set_index("ticker",inplace=True)

# This dataframe can then be used to query the CIK numbers of any set of desired (USA-based, publicly-traded) companies, 
# and use these CIK numbers for further API calls.

In [6]:
tickers_cik.head()

,cik_str,title
ticker,,
AAPL,0000320193,Apple Inc.
MSFT,0000789019,MICROSOFT CORP
BRK-B,0001067983,BERKSHIRE HATHAWAY INC
UNH,0000731766,UNITEDHEALTH GROUP INC
XOM,0000034088,EXXON MOBIL CORP


In [31]:
# Get Cisco, Apple, Microsoft, and IBM CIKs

#by index name
tickers_cik.loc[['AAPL','MSFT','CSCO','IBM']]

,cik_str,title
ticker,,
AAPL,0000320193,Apple Inc.
MSFT,0000789019,MICROSOFT CORP
CSCO,0000858877,"CISCO SYSTEMS, INC."
IBM,0000051143,INTERNATIONAL BUSINESS MACHINES CORP


In [32]:
# Get total assets of all four
response1 = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000858877/us-gaap/Assets.json", headers=headers)
response2 = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000789019/us-gaap/Assets.json", headers=headers)
response3 = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/Assets.json", headers=headers)
response4 = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000051143/us-gaap/Assets.json", headers=headers)

In [33]:
# View one of the objects
response1.json()

{'cik': 858877,
 'taxonomy': 'us-gaap',
 'tag': 'Assets',
 'label': 'Assets',
 'description': 'Sum of the carrying amounts as of the balance sheet date of all assets that are recognized. Assets are probable future economic benefits obtained or controlled by an entity as a result of past transactions or events.',
 'entityName': 'CISCO SYSTEMS, INC.',
 'units': {'USD': [{'end': '2009-07-25',
    'val': 68128000000,
    'accn': '0001193125-09-237055',
    'fy': 2010,
    'fp': 'Q1',
    'form': '10-Q',
    'filed': '2009-11-18'},
   {'end': '2009-07-25',
    'val': 68128000000,
    'accn': '0001193125-10-032740',
    'fy': 2010,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2010-02-17'},
   {'end': '2009-07-25',
    'val': 68128000000,
    'accn': '0001193125-10-128609',
    'fy': 2010,
    'fp': 'Q3',
    'form': '10-Q',
    'filed': '2010-05-26'},
   {'end': '2009-07-25',
    'val': 68128000000,
    'accn': '0001193125-10-213400',
    'fy': 2010,
    'fp': 'FY',
    'form': '10-K',


In [34]:
# Convert to Pandas DFs

assets_timeseries1 = pd.json_normalize(response1.json()["units"]["USD"])
assets_timeseries1["filed"] = pd.to_datetime(assets_timeseries1["filed"])
assets_timeseries1 = assets_timeseries1.sort_values("end")

assets_timeseries2 = pd.json_normalize(response2.json()["units"]["USD"])
assets_timeseries2["filed"] = pd.to_datetime(assets_timeseries2["filed"])
assets_timeseries2 = assets_timeseries2.sort_values("end")

assets_timeseries3 = pd.json_normalize(response3.json()["units"]["USD"])
assets_timeseries3["filed"] = pd.to_datetime(assets_timeseries3["filed"])
assets_timeseries3 = assets_timeseries3.sort_values("end")

assets_timeseries4 = pd.json_normalize(response4.json()["units"]["USD"])
assets_timeseries4["filed"] = pd.to_datetime(assets_timeseries4["filed"])
assets_timeseries4 = assets_timeseries4.sort_values("end")

In [35]:
# View one of the DFs
assets_timeseries1.tail(10)

,end,val,accn,fy,fp,form,filed,frame
100,2021-05-01,93896000000,0000858877-21-000008,2021,Q3,10-Q,2021-05-25,CY2021Q1I
101,2021-07-31,97497000000,0000858877-21-000013,2021,FY,10-K,2021-09-09,NaN
102,2021-07-31,97497000000,0000858877-21-000018,2022,Q1,10-Q,2021-11-23,NaN
103,2021-07-31,97497000000,0000858877-22-000004,2022,Q2,10-Q,2022-02-22,NaN
104,2021-07-31,97497000000,0000858877-22-000009,2022,Q3,10-Q,2022-05-25,NaN
105,2021-07-31,97497000000,0000858877-22-000013,2022,FY,10-K,2022-09-08,CY2021Q2I
106,2021-10-30,95981000000,0000858877-21-000018,2022,Q1,10-Q,2021-11-23,CY2021Q3I
107,2022-01-29,94262000000,0000858877-22-000004,2022,Q2,10-Q,2022-02-22,CY2021Q4I
108,2022-04-30,92797000000,0000858877-22-000009,2022,Q3,10-Q,2022-05-25,CY2022Q1I
109,2022-07-30,94002000000,0000858877-22-000013,2022,FY,10-K,2022-09-08,CY2022Q2I


In [38]:
# Visualize the data

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=assets_timeseries1["end"], y=assets_timeseries1["val"],
                    mode='lines+markers',
                    name='Cisco'))
fig.add_trace(go.Scatter(x=assets_timeseries2["end"], y=assets_timeseries2["val"],
                    mode='lines+markers',
                    name='Microsoft'))


fig.add_trace(go.Scatter(x=assets_timeseries3["end"], y=assets_timeseries3["val"],
                    mode='lines+markers',
                    name='Apple'))

fig.add_trace(go.Scatter(x=assets_timeseries4["end"], y=assets_timeseries4["val"],
                    mode='lines+markers',
                    name='IBM'))

fig.show()

In [39]:
# Put some titles on the graph

fig.update_layout(
    width=1000,
    height=700,
    paper_bgcolor='white',
    plot_bgcolor='#fafafa',
    hovermode='closest',
    title= "<b> Total Assets Comparison over Time </b>", title_x = .5, 
    xaxis = dict(
        title="Time"
    ),
    yaxis = dict(
        title="Total Assets value (USD)"
    ),  
    showlegend=True,

    annotations = [dict(xref='paper', yref='paper',
                                        x=0.5, y=-0.15,
                                        showarrow=False,
                                        text ='Data Source: SEC-EDGAR')]

)
    
fig.show()